In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("DataWrangling").getOrCreate()

In [14]:
#arch = "'C:\\Big Data\\movie_data_tmbd.csv'"
arch = "movie_data_tmbd.csv"

In [16]:
df = spark.read.format("csv").option("inferSchema", False).option("header", True).option("sep","|").load(arch)

In [18]:
df.printSchema()

root
 |-- adult: string (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string 

In [19]:
df.dtypes

[('adult', 'string'),
 ('backdrop_path', 'string'),
 ('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('genres', 'string'),
 ('homepage', 'string'),
 ('id', 'string'),
 ('imdb_id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('poster_path', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('spoken_languages', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('video', 'string'),
 ('vote_average', 'string'),
 ('vote_count', 'string'),
 ('cast', 'string'),
 ('directors', 'string')]

In [20]:
df.columns

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'cast',
 'directors']

In [21]:
df.count()

120127

In [27]:
cols = ['id','budget','popularity','release_date','revenue','title']

In [28]:
subconjunto = df.select(*cols)

In [29]:
subconjunto.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             3.892|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|             5.482|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|             8.262|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|              7.83|  1975-03-12|      0|          F for Fake|
|43004| 500000|             5.694|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             2.873|  1962-03-09|      0|           My Geisha|
|43007|      0|             3.433|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             7.869|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             3.775|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             7.185|  1962-01-01|      0|  Sodom and Gomorrah|

In [30]:
df.show()

+-----+--------------------+---------------------+-------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------+-------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+--------------------+--------------------+
|adult|       backdrop_path|belongs_to_collection| budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|        popularity|         poster_path|production_companies|production_countries|release_date|revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|                cast|           directors|
+-----+--------------------+---------------------+-------+--------------------+--------------------+-----+---------+-----------------+--

In [31]:
#Para calcular los valores faltantes en una columna o en varias utilizando funciones de PySpark
from pyspark.sql.functions import *

In [32]:
df.filter((df['popularity']=='')| df['popularity'].isNull()|isnan(df['popularity'])).count()

1059

In [34]:
#Calcular todos los valores faltantes en el DF
#alias gneera una nuevo dataset con una nuevo nombre ":c = df.alias("c") "
df.select([count(when((col(c)=='')|col(c).isNull()|isnan(c), c)).alias(c) for c in df.columns]).show()

+-----+-------------+---------------------+------+------+--------+---+-------+-----------------+--------------+--------+----------+-----------+--------------------+--------------------+------------+-------+-------+----------------+------+-------+-----+-----+------------+----------+----+---------+
|adult|backdrop_path|belongs_to_collection|budget|genres|homepage| id|imdb_id|original_language|original_title|overview|popularity|poster_path|production_companies|production_countries|release_date|revenue|runtime|spoken_languages|status|tagline|title|video|vote_average|vote_count|cast|directors|
+-----+-------------+---------------------+------+------+--------+---+-------+-----------------+--------------+--------+----------+-----------+--------------------+--------------------+------------+-------+-------+----------------+------+-------+-----+-----+------------+----------+----+---------+
|    0|        52968|               110564|   658|   658|   99919|658|   5575|              658|          

In [35]:
#Calcular las frecuencias por categoria(columna)
df.groupBy(df['title']).count().show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|   The Corn Is Green|    2|
|Meet The Browns -...|    1|
|Morenita, El Esca...|    1|
| Father Takes a Wife|    1|
|    Taking The Curve|    1|
|           Fleshburn|    1|
|          Dead Lenny|    1|
|El juego de la silla|    1|
|              Sargad|    1|
|                 Kin|    3|
|I Don't Feel at H...|    1|
|Penguins of Madag...|    1|
| Ormayundo Ee Mukham|    1|
|         Eat With Me|    1|
|  Ainda Há Pastores?|    1|
|The Werewolf of W...|    1|
|    Conquering China|    1|
|Discovering the R...|    1|
|My Wife Is a Gang...|    1|
|Depeche Mode: Tou...|    1|
+--------------------+-----+
only showing top 20 rows

